## Use ADQL and jointables to cross-match the Gaia sources in Koljonen et al. to ATNF and FAST

In [50]:
from astroquery.gaia import Gaia
import eliminating

In [51]:
# load in tables Gaia has
tables = Gaia.load_tables(only_names=True)

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


In [52]:
# see what tables Gaia has
for table in tables:
    print(table.name)

external.apassdr9
external.catwise2020
external.gaiadr2_astrophysical_parameters
external.gaiadr2_geometric_distance
external.gaiaedr3_distance
external.gaiaedr3_gcns_main_1
external.gaiaedr3_gcns_rejected_1
external.gaiaedr3_spurious
external.galex_ais
external.ravedr5_com
external.ravedr5_dr5
external.ravedr5_gra
external.ravedr5_on
external.ravedr6
external.sdssdr13_photoprimary
external.skymapperdr1_master
external.skymapperdr2_master
external.tmass_xsc
gaiadr1.aux_qso_icrf2_match
gaiadr1.ext_phot_zero_point
gaiadr1.allwise_best_neighbour
gaiadr1.allwise_neighbourhood
gaiadr1.gsc23_best_neighbour
gaiadr1.gsc23_neighbourhood
gaiadr1.ppmxl_best_neighbour
gaiadr1.ppmxl_neighbourhood
gaiadr1.sdss_dr9_best_neighbour
gaiadr1.sdss_dr9_neighbourhood
gaiadr1.tmass_best_neighbour
gaiadr1.tmass_neighbourhood
gaiadr1.ucac4_best_neighbour
gaiadr1.ucac4_neighbourhood
gaiadr1.urat1_best_neighbour
gaiadr1.urat1_neighbourhood
gaiadr1.cepheid
gaiadr1.phot_variable_time_series_gfov
gaiadr1.phot_varia

In [53]:
# get the metadata for gaiadr3.gaia_source
table_metadata = Gaia.load_table('gaiadr3.gaia_source')
print(table_metadata)

Retrieving table 'gaiadr3.gaia_source'


TAP Table name: gaiadr3.gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Num. columns: 152


In [54]:
# see what columns this table has
for column in table_metadata.columns:
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [55]:
# load in Koljonen source ids as a list
koljonen_sourceids = [455282205716288384, 3831382647922429952, 5367876720979404288, 3990037124929068032, 6128369984328414336, 6140785016794586752,
            6098156298150016768, 4358428942492430336, 6025344817107454464, 4059795674516044800, 6436867623955512064, 2115337192179377792,
            4519819661567533696, 6644467032871428992, 1834595731470345472, 6469722508861870080, 2672030065446134656, 2001168543319218048, 
            2440660623886405504, 4682464743003293312, 4656677385699742208, 2957031626919939456, 5645504747023158400, 705098703608575744,
            588191888537402112, 5203822684102798592, 5419965878188457984, 6096705840454620800, 6268529198286308224, 6496325574947304448,
            6179115508262195200, 6041127310076589056, 4379227476242700928, 4121864828231575168, 4526229058440076288, 4316237348443952128,
            1823773960079216896, 1872588462410154240, 544927450310303104, 588191888537402112, 1226507282368609152]

In [56]:
# write a gaia query to retrieve an astropy table of desired info for each source
query1_base = """
SELECT {columns}
FROM gaiadr3.gaia_source as gaia
WHERE (gaia.source_id = {source_id_cond})
"""

columns = 'source_id, ra, dec, parallax'

koljonen_sourceids_str = ' OR gaia.source_id = '.join(str(x) for x in koljonen_sourceids)

query1 = query1_base.format(columns=columns, source_id_cond=koljonen_sourceids_str)
print(query1)


SELECT source_id, ra, dec, parallax
FROM gaiadr3.gaia_source as gaia
WHERE (gaia.source_id = 455282205716288384 OR gaia.source_id = 3831382647922429952 OR gaia.source_id = 5367876720979404288 OR gaia.source_id = 3990037124929068032 OR gaia.source_id = 6128369984328414336 OR gaia.source_id = 6140785016794586752 OR gaia.source_id = 6098156298150016768 OR gaia.source_id = 4358428942492430336 OR gaia.source_id = 6025344817107454464 OR gaia.source_id = 4059795674516044800 OR gaia.source_id = 6436867623955512064 OR gaia.source_id = 2115337192179377792 OR gaia.source_id = 4519819661567533696 OR gaia.source_id = 6644467032871428992 OR gaia.source_id = 1834595731470345472 OR gaia.source_id = 6469722508861870080 OR gaia.source_id = 2672030065446134656 OR gaia.source_id = 2001168543319218048 OR gaia.source_id = 2440660623886405504 OR gaia.source_id = 4682464743003293312 OR gaia.source_id = 4656677385699742208 OR gaia.source_id = 2957031626919939456 OR gaia.source_id = 5645504747023158400 OR gaia

In [58]:
job1 = Gaia.launch_job(query1)
results1 = job1.get_results()
results1.show_in_notebook()

idx,source_id,ra,dec,parallax
,,deg,deg,mas
0,588191888537402112,143.83632970098836,9.009971735985376,0.7144949599085562
1,3831382647922429952,155.94870458335123,0.6446470547528212,0.6861708101572905
2,455282205716288384,33.043636404275425,53.36078082215321,0.8584448143420856
3,2957031626919939456,80.82054713445723,-25.460313037448238,0.4458118377520122
4,1834595731470345472,299.39420990475,25.267236202653383,2.1497858656760136
5,6096705840454620800,214.37735925120347,-44.04932767896711,0.19737278858599186
6,2115337192179377792,274.1497258625874,45.17606864561869,0.22249783718568827
7,3990037124929068032,162.18089987798868,23.664831000638443,0.48692630852515645
8,5645504747023158400,129.7100755896266,-28.465825811619446,0.4294249878990407


In [ ]:
# we next want to load in the ATNF pulsars and put it in a searchable format